## Kaggle data management api

In [ ]:
# !pip install kaggle # now download the api token and store it to /home/ec2-user/.kaggle/kaggle.json
# # !cd /home/ec2-user/SageMaker
# !mkdir /home/ec2-user/.kaggle/
# !mv /home/ec2-user/SageMaker/kaggle.json /home/ec2-user/.kaggle/kaggle.json
# !chmod 600 /home/ec2-user/.kaggle/kaggle.json # for privacy
!df -h --total  # get the harddisk info
!nvcc --version  # Get the cuda version


In [2]:
!git clone https://github.com/RamsteinWR/HaemorrhageRSNA19.git
# !rm -rf HaemorrhageRSNA19
import os
os.chdir("/home/ec2-user/SageMaker/")
# import torch
# print(torch.__version__)

Cloning into 'HaemorrhageRSNA19'...
remote: Enumerating objects: 379, done.
remote: Counting objects: 100% (379/379), done.
remote: Compressing objects: 100% (219/219), done.
remote: Total 379 (delta 231), reused 306 (delta 158), pack-reused 0
Receiving objects: 100% (379/379), 32.40 MiB | 24.74 MiB/s, done.
Resolving deltas: 100% (231/231), done.


In [1]:
%%time
# !kaggle competitions download -c rsna-intracranial-hemorrhage-detection
# mv /home/ec2-user/aptos2019-blindness-detection.zip /home/ec2-user/SageMaker/
# !unzip -o -q /home/ec2-user/SageMaker/rsna-intracranial-hemorrhage-detection.zip -d /home/ec2-user/SageMaker/rsna_dataset  #-q for quitely no verbose

# !zip -r -0 -q Haemorrhage_dataset.zip /home/ec2-user/SageMaker/Haemorrhage_dataset
# !pip install py7zr

# import py7zr
# with py7zr.SevenZipFile('Haemorrhage_dataset.7z', 'w') as archive:
#     archive.writeall('/home/ec2-user/SageMaker/Haemorrhage_dataset', 'base')

CPU times: user 7.84 ms, sys: 227 µs, total: 8.07 ms
Wall time: 357 ms


## Preprocessing the authenication management

In [1]:
%%time
import boto3
import botocore
from botocore.exceptions import ClientError
from tqdm import tqdm
import os, sys
import urllib.request
import re
import sagemaker
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri

role = get_execution_role()

bucket = "brain-imaging-intracranial-hemorrhage"
region_name="us-east-1"
aws_access_key_id = "gaudfguya"
aws_secret_access_key = "5haidufbauybfuy"

training_image = get_image_uri(boto3.Session().region_name, 'image-classification')

def download(url):
    filename = url.split("/")[-1]
    if not os.path.exists(filename):
        urllib.request.urlretrieve(url, filename)

def upload_to_s3(channel, file):
    s3 = boto3.resource('s3',
                             region_name=region_name,
                             aws_access_key_id=aws_access_key_id,
                             aws_secret_access_key=aws_secret_access_key)    
    data = open(file, "rb")
    key = channel + '/' + file
    print("Uploading file {} to s3://{}/{}".format(file, bucket, channel))
    s3.Bucket(bucket).put_object(Key=key, Body=data)
    
def download(url):
    filename = url.split("/")[-1]
    if not os.path.exists(filename):
        urllib.request.urlretrieve(url, filename)

def upload_to_s3(channel, file):
    s3 = boto3.resource('s3')
    data = open(file, "rb")
    key = channel + '/' + file
    s3.Bucket(bucket).put_object(Key=key, Body=data)

def upload_dir_to_s3(bucket, s3_folder, dir_to_upload):
    s3_client = boto3.client('s3',
                             region_name=region_name,
                             aws_access_key_id=aws_access_key_id,
                             aws_secret_access_key=aws_secret_access_key)
    print("Uploading {} to s3://{}/{}".format(dir_to_upload, bucket, s3_folder))
    # enumerate local files recursively
    for root, dirs, files in os.walk(dir_to_upload):
        for filename in tqdm(files):
            # construct the full local path
            local_path = os.path.join(root, filename)
            # construct the full Dropbox path
            relative_path = os.path.relpath(local_path, dir_to_upload)
            s3_path = os.path.join(s3_folder, relative_path).replace("\\", "/")
            try:
                s3_client.head_object(Bucket=bucket, Key=s3_path)
                print("Path found on S3! Deleting %s..." % s3_path)
                try:
                    s3_client.delete_object(Bucket=bucket, Key=s3_path)
                    try:
#                         print("Uploading {} to s3://{}/{}".format(dir_to_upload, bucket, s3_path)
                        s3_client.upload_file(local_path, Bucket=bucket, Key=s3_path)
                    except ClientError as e:
                        logging.error(e)
                except:
                    print("Unable to delete from s3 %s..." % s3_path)
            except:
                try:
                    s3_client.upload_file(local_path, Bucket=bucket, Key=s3_path)
                except ClientError as e:
                    logging.error(e)
    print("Upload completed successfully.")
    
def download_dir(s3_folder, local_path, bucket=""):
    """
    params:
    - s3_folder: pattern to match in s3
    - local_path: local_path path to folder in which to place files
    - bucket: s3 bucket with target contents
    - client: initialized s3 client object
    """
    client = boto3.client('s3', region_name=region_name)
    keys = []
    dirs = []
    next_token = ''
    base_kwargs = {
        'Bucket': bucket,
        'Prefix': s3_folder,
    }
    while next_token is not None:
        kwargs = base_kwargs.copy()
        if next_token != '':
            kwargs.update({'ContinuationToken': next_token})
        results = client.list_objects_v2(**kwargs)
        contents = results.get('Contents')
        for i in contents:
            k = i.get('Key')
            if k[-1] != '/':
                keys.append(k)
            else:
                dirs.append(k)
        next_token = results.get('NextContinuationToken')
    for d in dirs:
        dest_pathname = os.path.join(local_path, d)
        if not os.path.exists(os.path.dirname(dest_pathname)):
            os.makedirs(os.path.dirname(dest_pathname))
    print("{} files found in {} directories. Downloading now...".format(len(keys), len(dirs)))
    for k in tqdm(keys):
        dest_pathname = os.path.join(local_path, k)
        if not os.path.exists(os.path.dirname(dest_pathname)):
            os.makedirs(os.path.dirname(dest_pathname))
        try:
#             print("Downloading {}".format(dest_pathname))
            client.download_file(bucket, k, dest_pathname)
        except botocore.exceptions.ClientError as e:
            if e.response['Error']['Code'] == "404":
                print("The object does not exist.")
            else:
                raise
    print("{} files downloaded successfully.".format(len(keys)))
    
# download('http://data.lip6.fr/cadene/pretrained/home/ec2-user/SageMaker/rsna19/models/se_resnext50_32x4d-a260b3a4.pth')
# upload_to_s3("pretrained", 'se_resnext50_32x4d-a260b3a4.pth')
# upload_dir_to_s3(bucket=bucket, s3_folder='aptos-2019', dir_to_upload="/home/ec2-user/SageMaker/data")

# download_dir(s3_folder='aptos-2015/train_images_768/', local_path='/home/ec2-user/SageMaker/data/aptos-2015/train_images_768/', bucket=bucket)
# download_dir(s3_folder='aptos-2015/test_images_768/', local_path='/home/ec2-user/SageMaker/data/aptos-2015/test_images_768/', bucket=bucket)

The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


CPU times: user 3.04 s, sys: 3.49 s, total: 6.53 s
Wall time: 9.43 s


In [2]:
# upload_dir_to_s3(bucket=bucket, s3_folder='Haemorrhage_dataset', dir_to_upload="/home/ec2-user/SageMaker/Haemorrhage_dataset")
# upload_to_s3("Haemorrhage_dataset", 'home/ec2-user/SageMaker/Haemorrhage_dataset.zip')


## Uploading files to s3 fast with sagemaker.s3.S3Uploader()

<a id='idg4c.1'></a>
## Dependencies
___
### import packages and check SageMaker version

In [3]:
import json
import torch
import tarfile
import pickle
import matplotlib.pyplot as plt
import torchvision as tv
import pathlib                          # Path management tool (standard library)
import subprocess                       # Runs shell commands via Python (standard library)
import sagemaker                        # SageMaker Python SDK
from sagemaker.pytorch import PyTorch   # PyTorch Estimator for TensorFlow

In [4]:
!pip install -r /home/ec2-user/SageMaker/HaemorrhageRSNA19/requirements.txt

  Cloning https://github.com/NVIDIA/apex to /tmp/pip-req-build-3nq5ake9
     |████████████████████████████████| 72 kB 809 kB/s  eta 0:00:01
     |████████████████████████████████| 3.3 MB 23.0 MB/s eta 0:00:01
     |████████████████████████████████| 58 kB 8.8 MB/s s eta 0:00:01
     |████████████████████████████████| 1.9 MB 106.1 MB/s eta 0:00:01
     |████████████████████████████████| 671 kB 104.7 MB/s eta 0:00:01
     |████████████████████████████████| 66 kB 908 kB/s s eta 0:00:01
     |█████████████▎                  | 322.1 MB 135.1 MB/s eta 0:00:04

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



     |███████████████████████████▎    | 663.3 MB 148.2 MB/s eta 0:00:01

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



     |████████████████████████████████| 776.8 MB 10 kB/s 
     |████████████████████████████████| 7.6 MB 42.8 MB/s eta 0:00:01
     |████████████████████████████████| 12.8 MB 34.3 MB/s eta 0:00:01
     |████████████████████████████████| 62 kB 2.4 MB/s s eta 0:00:01
     |████████████████████████████████| 103.4 MB 77 kB/s s eta 0:00:01
     |████████████████████████████████| 98 kB 13.1 MB/s  eta 0:00:01
     |████████████████████████████████| 948 kB 93.9 MB/s eta 0:00:01
     |████████████████████████████████| 37.6 MB 36.4 MB/s eta 0:00:01
     |████████████████████████████████| 1.0 MB 102.8 MB/s eta 0:00:01
     |████████████████████████████████| 10.6 MB 81.9 MB/s eta 0:00:01
     |████████████████████████████████| 127 kB 118.7 MB/s eta 0:00:01
     |████████████████████████████████| 114 kB 141.9 MB/s eta 0:00:01
     |████████████████████████████████| 4.0 MB 129.7 MB/s eta 0:00:01
     |████████████████████████████████| 96 kB 9.1 MB/s s eta 0:00:01
     |██████████████████████████████

## Data conversion
### Appending module HaemorrhageRSNA19 to sys.path

In [3]:
import os, sys
os.chdir("/home/ec2-user/SageMaker/")
sys.path.append("/home/ec2-user/SageMaker/HaemorrhageRSNA19")
sys.path.append("/home/ec2-user/SageMaker/HaemorrhageRSNA19/configs")
sys.path.append("/home/ec2-user/SageMaker/HaemorrhageRSNA19/data")
sys.path.append("/home/ec2-user/SageMaker/HaemorrhageRSNA19/models")
sys.path.append("/home/ec2-user/SageMaker/HaemorrhageRSNA19/preprocessing")

print(sys.path)

['', '/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python36.zip', '/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6', '/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/lib-dynload', '/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages', '/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages/IPython/extensions', '/home/ec2-user/.ipython', '/home/ec2-user/SageMaker/HaemorrhageRSNA19', '/home/ec2-user/SageMaker/HaemorrhageRSNA19/configs', '/home/ec2-user/SageMaker/HaemorrhageRSNA19/data', '/home/ec2-user/SageMaker/HaemorrhageRSNA19/models', '/home/ec2-user/SageMaker/HaemorrhageRSNA19/preprocessing']


In [4]:
# %%time
# # Generate data index and save it as .pkl
# os.chdir("/home/ec2-user/SageMaker/HaemorrhageRSNA19")

# # !python create_dataframe.py

# !python create_symlinks.py


In [5]:
%%time
# Convert dicom images (slices) to npy arrays and pngs images
os.chdir("/home/ec2-user/SageMaker/HaemorrhageRSNA19")

!python convert_dataset.py
# !python /home/ec2-user/SageMaker/HaemorrhageRSNA19/data/scripts/prepare_3d_data.py



  2%|▋                               | 91903/3923635 [08:26<5:28:26, 194.44it/s]^C
Traceback (most recent call last):
  File "convert_dataset.py", line 35, in convert_sample
    save_image(path, img, img_orig_hu)
  File "convert_dataset.py", line 74, in save_image
    cv2.imwrite(dst_path, img)
KeyboardInterrupt
Traceback (most recent call last):
  File "convert_dataset.py", line 35, in convert_sample
    save_image(path, img, img_orig_hu)
  File "convert_dataset.py", line 74, in save_image
    cv2.imwrite(dst_path, img)
KeyboardInterrupt
Traceback (most recent call last):
  File "convert_dataset.py", line 34, in convert_sample
    draw_labels(path, img)
  File "convert_dataset.py", line 43, in draw_labels
    scan_id = pydicom.dcmread(path).SOPInstanceUID
  File "/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages/pydicom/filereader.py", line 888, in dcmread
    force=force, specific_tags=specific_tags)
  File "/home/ec2-user/anaconda3/envs/pytorch_latest_p36/

In [2]:
# import shutil
# shutil.rmtree('/home/ec2-user/SageMaker/Haemorrhage_dataset/trainOld', ignore_errors=True)

KeyboardInterrupt: 

## Training models 1/2 (Dmytro's models)

In [ ]:
# For training stage 1 models run the following commands for folds 0-4:

!python /home/ec2-user/SageMaker/rsna19/models/clf2D/train.py train --model resnet18_400 --fold 0
# !python /home/ec2-user/SageMaker/rsna19/models/clf2D/train.py train --model resnet34_400_5_planes_combine_last_var_dr0 --fold 0
# !python /home/ec2-user/SageMaker/rsna19/models/clf2D/train_segmentation.py train --model resnet18_384_5_planes_bn_f8 --fold 0
# !python /home/ec2-user/SageMaker/rsna19/models/clf3D/train_3d.py train --model dpn68_384_5_planes_combine_last --fold 0
# !python /home/ec2-user/SageMaker/rsna19/models/clf2D/train.py train --model airnet50_384 --fold 0 --apex


# For stage 2 models run the following for folds 0-4:

# !python /home/ec2-user/SageMaker/rsna19/models/clf2D/train.py train --model se_preresnext26b_400 --fold 0 --apex
# !python /home/ec2-user/SageMaker/rsna19/models/clf2D/train.py train --model resnext50_400 --fold 0 --apex

## Generating predictions for challenge data 1/2 (Dmytro's models)
### Run the following set of commands for folds 0-4:

In [ ]:
# Stage 1 models out-of-fold predictions
!python /home/ec2-user/SageMaker/rsna19/models/clf2D/predict.py predict_oof --model resnet18_400 --epoch 6 --fold 0 --mode all
# !python /home/ec2-user/SageMaker/rsna19/models/clf2D/predict.py predict_oof --model resnet34_400_5_planes_combine_last_var_dr0 --epoch 7 --fold 0 --mode all
# !python /home/ec2-user/SageMaker/rsna19/models/clf3D/predict.py predict_oof --model dpn68_384_5_planes_combine_last --epoch 72 --fold 0 --mode all
# !python /home/ec2-user/SageMaker/rsna19/models/clf2D/predict.py predict_oof --model resnet18_384_5_planes_bn_f8 --epoch 6 --fold 0 --mode all
# !python /home/ec2-user/SageMaker/rsna19/models/clf2D/predict.py predict_oof --model airnet50_384 --epoch 6 --fold 0 --mode all

# Stage 1 models test predictions
!python /home/ec2-user/SageMaker/rsna19/models/clf2D/predict.py predict_test --model resnet18_400 --epoch 6 --fold 0 --mode all
# !python /home/ec2-user/SageMaker/rsna19/models/clf2D/predict.py predict_test --model resnet34_400_5_planes_combine_last_var_dr0 --epoch 7 --fold 0 --mode all
# !python /home/ec2-user/SageMaker/rsna19/models/clf3D/predict.py predict_test --model dpn68_384_5_planes_combine_last --epoch 72 --fold 0 --mode all
# !python /home/ec2-user/SageMaker/rsna19/models/clf2D/predict.py predict_test --model resnet18_384_5_planes_bn_f8 --epoch 6 --fold 0 --mode all
# !python /home/ec2-user/SageMaker/rsna19/models/clf2D/predict.py predict_test --model airnet50_384 --epoch 6 --fold 0 --mode all

# # Stage 2 models out-of-fold predictions
# !python /home/ec2-user/SageMaker/rsna19/models/clf2D/predict.py predict_oof --model se_preresnext26b_400 --epoch 6 --fold 0 --mode all
# !python /home/ec2-user/SageMaker/rsna19/models/clf2D/predict.py predict_oof --model resnext50_400 --epoch 6 --fold 0 --mode all

# # Stage 2 models test predictions
# !python /home/ec2-user/SageMaker/rsna19/models/clf2D/predict.py predict_test --model se_preresnext26b_400 --epoch 6 --fold 0 --mode all
# !python /home/ec2-user/SageMaker/rsna19/models/clf2D/predict.py predict_test --model resnext50_400 --epoch 6 --fold 0 --mode all